### Deep Learning Assignment 3(regularization)

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

##### First reload the data we generated in notmist.ipynb.

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set:','train_dataset.shape', train_dataset.shape, ', train_labels.shape',train_labels.shape
  print 'Validation set:','valid_dataset.shape', valid_dataset.shape,', valid_labels.shape', valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape  # original : 18724


Training set: train_dataset.shape (200000, 28, 28) , train_labels.shape (200000,)
Validation set: valid_dataset.shape (10000, 28, 28) , valid_labels.shape (10000,)
Test set (10000, 28, 28) (10000,)


####  Reformat into a shape that's more adapted to the models we're going to train:

    data as a flat matrix(2d array) by  dataset.reshape((-1, image_size*image_size)).astype(np.float32),
    labels as float 1-hot encodings.



In [8]:
## different udacity version which test_dataset.shape(18724, 28, 28) test_labels.shape(18724,)
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32) #convert 2d array from 3d
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)  #1-hot encoding
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape  #original 18724

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### Problem 1

Introduce and tune <span style="color:red">L2 regularization </span>for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor t using<span style="color:red"> nn.l2_loss(t)</span>. The right amount of regularization should improve your validation / test accuracy.


In [17]:
#test
batch_size = 128
num_steps = 3001
offset_list=[]
print "train_labels.shape[0]:",train_labels.shape[0]
for step in xrange(num_steps):
   offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
   offset_list.append(offset)
print offset_list[0:10]
print offset_list[-10:-1]

train_labels.shape[0]: 200000
[0, 128, 256, 384, 512, 640, 768, 896, 1024, 1152]
[182976, 183104, 183232, 183360, 183488, 183616, 183744, 183872, 184000]


In [11]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Define regularization
    # L2 amounts to adding a penalty on the norm of the weights to the loss
  regularization = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.001 * regularization
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
#train
def train(num_steps):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
    #1st [0:128],2nd offset:((1,2,3..3001)*128) % (200000-128),...   
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print "Minibatch loss at step", step, ":", l
        print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
        print "Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

train(3001)

Initialized
Minibatch loss at step 0 : 20.8532
Minibatch accuracy: 7.8%
Validation accuracy: 11.1%
Minibatch loss at step 500 : 2.953
Minibatch accuracy: 75.8%
Validation accuracy: 76.5%
Minibatch loss at step 1000 : 2.05176
Minibatch accuracy: 73.4%
Validation accuracy: 77.9%
Minibatch loss at step 1500 : 0.953579
Minibatch accuracy: 84.4%
Validation accuracy: 79.4%
Minibatch loss at step 2000 : 0.750201
Minibatch accuracy: 85.2%
Validation accuracy: 80.7%
Minibatch loss at step 2500 : 0.732185
Minibatch accuracy: 88.3%
Validation accuracy: 81.4%
Minibatch loss at step 3000 : 0.918553
Minibatch accuracy: 75.8%
Validation accuracy: 81.2%
Test accuracy: 88.4%


#### Problem 2

Let's demonstrate an extreme case of <span style="color:red">overfitting</span>. Restrict your training data to just a few batches. What happens?
In [7]:



In [18]:
train(100)

Initialized
Minibatch loss at step 0 : 17.9859
Minibatch accuracy: 13.3%
Validation accuracy: 19.4%
Test accuracy: 77.8%


#### Problem 3

Introduce <span style="color:red">Dropout on the hidden layer</span> of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides<span style="color:red"> nn.dropout()</span> for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [19]:
num_hidden = 1024
# Create a one-hidden-layer network with dropout
def one_layer_dropout():
  batch_size = 128
  graph = tf.Graph()
  with graph.as_default():
    # Define placeholders of data and dropout_prob
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    dropout_p = tf.placeholder(tf.float32)

    # Define weight and bias terms
    W_in = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
    b_in = tf.Variable(tf.zeros([num_hidden]))

    W_hidden = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    b_hidden = tf.Variable(tf.zeros([num_labels]))

    # Define computation and predictions
    # Train
    first_layer = tf.add(tf.matmul(train_X, W_in), b_in)
    relu_layer = tf.nn.relu(first_layer)
    dropout_layer = tf.nn.dropout(relu_layer, dropout_p)
    second_layer = tf.add(tf.matmul(dropout_layer, W_hidden), b_hidden)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(second_layer, train_y))
    train_prediction = tf.nn.softmax(second_layer)

    # Valid
    v_first_layer = tf.add(tf.matmul(valid_X, W_in), b_in)
    v_relu_layer = tf.nn.relu(v_first_layer)
    v_second_layer = tf.add(tf.matmul(v_relu_layer, W_hidden), b_hidden)
    valid_prediction = tf.nn.softmax(v_second_layer)

    # Test
    t_first_layer = tf.add(tf.matmul(test_X, W_in), b_in)
    t_relu_layer = tf.nn.relu(t_first_layer)
    t_second_layer = tf.add(tf.matmul(t_relu_layer, W_hidden), b_hidden)
    test_prediction = tf.nn.softmax(t_second_layer)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

  # Let's run it:

  num_steps = 3001

  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {train_X : batch_data, train_y : batch_labels, dropout_p : 0.3}
      _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print "Minibatch loss at step", step, ":", l
        print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
        print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

one_layer_dropout()

Initialized
Minibatch loss at step 0 : 590.241
Minibatch accuracy: 13.3%
Validation accuracy: 11.3%
Minibatch loss at step 500 : 78.2559
Minibatch accuracy: 78.9%
Validation accuracy: 80.3%
Minibatch loss at step 1000 : 115.588
Minibatch accuracy: 68.0%
Validation accuracy: 81.9%
Minibatch loss at step 1500 : 45.2509
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 2000 : 36.1798
Minibatch accuracy: 77.3%
Validation accuracy: 83.1%
Minibatch loss at step 2500 : 43.5344
Minibatch accuracy: 77.3%
Validation accuracy: 83.2%
Minibatch loss at step 3000 : 50.0074
Minibatch accuracy: 71.1%
Validation accuracy: 83.4%
Test accuracy: 90.2%


#### Problem 4

Try to get the best performance you can using a<span style="color:red"> multi-layer model</span>! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use<span style="color:red"> learning rate decay</span>:

global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

 

In [20]:
# Create a deep network with dropout
def deep_dropout():
  batch_size = 128
  n_hidden_1 = 2048
  n_hidden_2 = 2048

  graph = tf.Graph()
  with graph.as_default():
    # Define placeholders of data and dropout_prob
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    dropout_p1 = tf.placeholder(tf.float32)
    dropout_p2 = tf.placeholder(tf.float32)

    # Define weight and bias terms
    W_in = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1], stddev=0.03))
    b_in = tf.Variable(tf.zeros([n_hidden_1]))

    W_hidden_1 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.03))
    b_hidden_1 = tf.Variable(tf.zeros([n_hidden_2]))

    W_out = tf.Variable(tf.truncated_normal([n_hidden_2, num_labels], stddev=0.03))
    b_out = tf.Variable(tf.zeros([num_labels]))

    # Define computation and predictions
    # Train
    layer1 = tf.add(tf.matmul(train_X, W_in), b_in)
    relu1 = tf.nn.relu(layer1)
    dropout1 = tf.nn.dropout(relu1, dropout_p1)
    l2_loss_1 = tf.nn.l2_loss(W_in) + tf.nn.l2_loss(b_in)  #same regularization above 2 cell

    layer2 = tf.add(tf.matmul(dropout1, W_hidden_1), b_hidden_1)
    relu2 = tf.nn.relu(layer2)
    dropout2 = tf.nn.dropout(relu2, dropout_p2)
    l2_loss_2 = tf.nn.l2_loss(W_hidden_1) + tf.nn.l2_loss(b_hidden_1)

    out = tf.add(tf.matmul(dropout2, W_out), b_out)
    l2_loss_3 = tf.nn.l2_loss(W_out) + tf.nn.l2_loss(b_out)
    
    # Regularization
    regularization = l2_loss_1 + l2_loss_2 + l2_loss_3
    
    # Not adding regularization for now, but you can add it to loss as
    # loss = loss + 0.0001 * regularization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, train_y))
    train_prediction = tf.nn.softmax(out)

    # Valid
    v_layer1 = tf.add(tf.matmul(valid_X, W_in), b_in)
    v_relu1 = tf.nn.relu(v_layer1)

    v_layer2 = tf.add(tf.matmul(v_relu1, W_hidden_1), b_hidden_1)
    v_relu2 = tf.nn.relu(v_layer2)

    v_out = tf.add(tf.matmul(v_relu2, W_out), b_out)
    valid_prediction = tf.nn.softmax(v_out)

    # Test
    t_layer1 = tf.add(tf.matmul(test_X, W_in), b_in)
    t_relu1 = tf.nn.relu(t_layer1)

    t_layer2 = tf.add(tf.matmul(t_relu1, W_hidden_1), b_hidden_1)
    t_relu2 = tf.nn.relu(t_layer2)

    t_out = tf.add(tf.matmul(t_relu2, W_out), b_out)
    test_prediction = tf.nn.softmax(t_out)

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

  # Let's run it:

  num_steps = 12001

  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {train_X : batch_data, train_y : batch_labels,
                   dropout_p1 : 0.75, dropout_p2: 0.9}
      _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print "Minibatch loss at step", step, ":", l
        print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
        print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

deep_dropout()

Initialized
Minibatch loss at step 0 : 2.37131
Minibatch accuracy: 5.5%
Validation accuracy: 20.0%
Minibatch loss at step 500 : 0.480525
Minibatch accuracy: 86.7%
Validation accuracy: 85.2%
Minibatch loss at step 1000 : 0.579251
Minibatch accuracy: 82.8%
Validation accuracy: 86.5%
Minibatch loss at step 1500 : 0.299408
Minibatch accuracy: 95.3%
Validation accuracy: 87.5%
Minibatch loss at step 2000 : 0.298626
Minibatch accuracy: 92.2%
Validation accuracy: 88.3%
Minibatch loss at step 2500 : 0.235718
Minibatch accuracy: 93.0%
Validation accuracy: 88.7%
Minibatch loss at step 3000 : 0.431049
Minibatch accuracy: 85.9%
Validation accuracy: 89.0%
Minibatch loss at step 3500 : 0.341678
Minibatch accuracy: 90.6%
Validation accuracy: 89.4%
Minibatch loss at step 4000 : 0.287691
Minibatch accuracy: 93.0%
Validation accuracy: 89.2%
Minibatch loss at step 4500 : 0.278906
Minibatch accuracy: 92.2%
Validation accuracy: 89.6%
Minibatch loss at step 5000 : 0.272939
Minibatch accuracy: 92.2%
Validatio